# Análisis Juegos Olímpicos: URSS, Rusia y USA

**Dataset:** Olympic Games 1896-2024

**Fecha:** Enero 2026

---


## 1. Imports


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## 2. Carga de Datos


In [ ]:
jjoo = pd.read_csv("athlete_events.csv")
print(f"Dataset cargado: {len(jjoo):,} registros")


## 3. Tratamiento de Nulos


In [ ]:
jjoo["Medal"] = jjoo["Medal"].fillna("NoM")


In [ ]:
jjoo["Height"] = jjoo.groupby(["Sex"])['Height'].transform(lambda x: x.fillna(x.median()))
jjoo["Age"] = jjoo.groupby(["Sex"])['Age'].transform(lambda x: x.fillna(x.median()))
jjoo["Weight"] = jjoo.groupby(["Sex"])['Weight'].transform(lambda x: x.fillna(x.median()))


## 4. Añadir Columna Country


In [ ]:
noc = pd.read_csv('noc_regions.csv')
noc = noc.drop("notes", axis=1)
noc = noc.rename(columns={"region": "Country"})


In [ ]:
jjoo_merge = jjoo.merge(noc, left_on="NOC", right_on="NOC", how="left")


In [ ]:
jjoo = jjoo_merge[jjoo_merge["Season"] == "Summer"]


In [ ]:
jjoo.loc[jjoo['NOC']=='SGP', "Country"] = "Singapore"
jjoo.loc[jjoo['NOC']=='ROT', "Country"] = "Refugee Olympic Athletes"
jjoo.loc[jjoo['NOC']=='UNK', "Country"] = "Unknown"
jjoo.loc[jjoo['NOC']=='TUV', "Country"] = "Tuvalu"
jjoo = jjoo.drop(["Team"], axis=1)


## 5. Crear Columna Has_Medal


In [ ]:
jjoo["Has_Medal"] = "0"
jjoo.loc[jjoo["Medal"] != "NoM", "Has_Medal"] = 1


## 6. Identificar Deportes de Equipo


In [ ]:
medal_sum = jjoo.groupby(["Country","Year","Event","Medal"], as_index=False)["Has_Medal"].sum()
gold_medals = medal_sum[medal_sum['Medal'] == 'Gold']
identify_team_events = gold_medals[gold_medals['Has_Medal'] > 1]


In [ ]:
team_event = []
for event in jjoo["Event"]:
    if event in identify_team_events["Event"].values:
        team_event.append(True)
    else:
        team_event.append(False)

medal = jjoo['Has_Medal'] == 1
jjoo['Team_Event'] = 0
jjoo.loc[team_event & medal, "Team_Event"] = 1


## 7. Crear medals_tally (Limpieza de Duplicados)


In [ ]:
medals_tally = jjoo[jjoo["Has_Medal"] == 1].groupby(["Year", "Country", "Event", "Medal"]).agg(
    atletas=("Has_Medal","sum"),
    Team_Event=("Team_Event","max")
).reset_index()

medals_tally['Medal_corrected'] = 1
print(f"✅ medals_tally creado: {len(medals_tally):,} medallas")


## 8. Filtrado de Países (URSS, Rusia, USA)


In [ ]:
# Filtrar DIRECTAMENTE desde jjoo (no desde medals_tally)
# Esto elimina duplicados correctamente

urss = jjoo[
    (jjoo['NOC'] == 'URS') & 
    (jjoo['Year'] >= 1952) & 
    (jjoo['Year'] <= 1988) &
    (jjoo['Has_Medal'] == 1)
].drop_duplicates(subset=['Year', 'NOC', 'Event', 'Medal']).copy()

rus = jjoo[
    (jjoo['NOC'] == 'RUS') & 
    (jjoo['Year'] >= 1996) &
    (jjoo['Has_Medal'] == 1)
].drop_duplicates(subset=['Year', 'NOC', 'Event', 'Medal']).copy()

usa = jjoo[
    (jjoo['NOC'] == 'USA') &
    (jjoo['Has_Medal'] == 1)
].drop_duplicates(subset=['Year', 'NOC', 'Event', 'Medal']).copy()

print(f"✅ URSS: {len(urss):,} medallas")
print(f"✅ Rusia: {len(rus):,} medallas")
print(f"✅ USA: {len(usa):,} medallas")


---

# ANÁLISIS Y GRÁFICOS

---


## GRÁFICO 1: Evolución Temporal URSS (1952-1988)


In [ ]:
medallas_urss_año = urss.groupby('Year').size()

fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(medallas_urss_año.index, medallas_urss_año.values, marker='o', linewidth=3, markersize=10, color='#d62728', alpha=0.8)

año_max = medallas_urss_año.idxmax()
valor_max = medallas_urss_año.max()
ax.scatter(año_max, valor_max, color='gold', s=400, zorder=5, edgecolors='black', linewidths=2)
ax.annotate(f'{año_max}\\n{valor_max} medallas', xy=(año_max, valor_max), xytext=(año_max, valor_max + 15),
            arrowprops=dict(arrowstyle='->', color='gold', lw=2), fontsize=11, ha='center', fontweight='bold', color='darkred')

if 1980 in medallas_urss_año.index:
    ax.axvline(x=1980, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax.text(1980, ax.get_ylim()[1] * 0.95, 'Boicot USA', ha='center', fontsize=10, fontweight='bold', color='red')

ax.set_xlabel('Año Olímpico', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Evolución Temporal: Medallas URSS por Olimpiada (1952-1988)', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xticks(medallas_urss_año.index)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print(f'Total: {len(urss):,} medallas | Promedio: {medallas_urss_año.mean():.1f} medallas/olimpiada')


## GRÁFICO 2: Evolución Temporal USA (1896-2024)


In [ ]:
medallas_usa_año = usa.groupby('Year').size()

fig, ax = plt.subplots(figsize=(16, 7))
ax.plot(medallas_usa_año.index, medallas_usa_año.values, marker='o', linewidth=2.5, markersize=7, color='#1f77b4', alpha=0.8)

if 1904 in medallas_usa_año.index:
    ax.scatter(1904, medallas_usa_año[1904], color='orange', s=300, zorder=5, edgecolors='black', linewidths=2)
    ax.annotate('1904: St. Louis\\nVentaja local', xy=(1904, medallas_usa_año[1904]), xytext=(1904, medallas_usa_año[1904] + 30),
                arrowprops=dict(arrowstyle='->', color='orange', lw=2), fontsize=10, ha='center', fontweight='bold', color='darkorange')

ax.scatter(1980, 0, color='red', s=300, marker='x', linewidths=4, zorder=5)
ax.annotate('1980: Moscú\\nBoicot USA', xy=(1980, 0), xytext=(1980, 40),
            arrowprops=dict(arrowstyle='->', color='red', lw=2), fontsize=10, ha='center', fontweight='bold', color='red')

if 1984 in medallas_usa_año.index:
    ax.scatter(1984, medallas_usa_año[1984], color='gold', s=300, zorder=5, edgecolors='black', linewidths=2)
    ax.annotate('1984: Los Ángeles\\nSin URSS', xy=(1984, medallas_usa_año[1984]), xytext=(1984, medallas_usa_año[1984] + 30),
                arrowprops=dict(arrowstyle='->', color='gold', lw=2), fontsize=10, ha='center', fontweight='bold', color='darkgoldenrod')

ax.axvspan(1979.5, 1980.5, alpha=0.2, color='red', label='Boicot USA')
ax.axvspan(1983.5, 1984.5, alpha=0.2, color='blue', label='Boicot URSS')

ax.set_xlabel('Año Olímpico', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Evolución Temporal: Medallas USA por Olimpiada (1896-2024)', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=10, loc='upper left')
ax.set_xlim(1895, 2025)
plt.tight_layout()
plt.show()


## GRÁFICO 3: Comparación Promedio URSS vs Rusia


In [ ]:
medallas_rus_año = rus.groupby('Year').size()
promedio_urss = urss.groupby('Year').size().mean()
promedio_rus = medallas_rus_año.mean()
total_urss = len(urss)
total_rus = len(rus)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Total
bars1 = axes[0].bar(['URSS\\n(1952-1988)', 'Rusia\\n(1996-2024)'], [total_urss, total_rus], 
                    color=['#d62728', '#1f77b4'], alpha=0.8, edgecolor='black', linewidth=2, width=0.6)
axes[0].set_ylabel('Medallas Totales', fontsize=12, fontweight='bold')
axes[0].set_title('Total de Medallas', fontsize=14, fontweight='bold', pad=15)
axes[0].grid(True, axis='y', alpha=0.3, linestyle='--')
for bar, val in zip(bars1, [total_urss, total_rus]):
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 20, f'{val:,}', 
                ha='center', va='bottom', fontsize=13, fontweight='bold')

# Promedio
bars2 = axes[1].bar(['URSS', 'Rusia'], [promedio_urss, promedio_rus], 
                    color=['#d62728', '#1f77b4'], alpha=0.8, edgecolor='black', linewidth=2, width=0.6)
axes[1].set_ylabel('Medallas por Olimpiada', fontsize=12, fontweight='bold')
axes[1].set_title('Promedio de Medallas por Olimpiada', fontsize=14, fontweight='bold', pad=15)
axes[1].grid(True, axis='y', alpha=0.3, linestyle='--')
for bar, val in zip(bars2, [promedio_urss, promedio_rus]):
    axes[1].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 2, f'{val:.1f}', 
                ha='center', va='bottom', fontsize=13, fontweight='bold')

plt.suptitle('Comparación: URSS vs Rusia', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f'Caída: -{((promedio_urss - promedio_rus) / promedio_urss * 100):.1f}% en promedio')


## GRÁFICO 4: Top 10 Deportes - URSS vs Rusia


In [ ]:
top_deportes_urss = urss.groupby('Sport').size().sort_values(ascending=False).head(10)
deportes_top = top_deportes_urss.index.tolist()
top_deportes_rus = rus[rus['Sport'].isin(deportes_top)].groupby('Sport').size()
top_deportes_rus = top_deportes_rus.reindex(deportes_top, fill_value=0)

fig, ax = plt.subplots(figsize=(14, 7))
x = np.arange(len(deportes_top))
ancho = 0.35

ax.bar(x - ancho/2, top_deportes_urss.values, ancho, label='URSS (1952-1988)', color='#d62728', alpha=0.8)
ax.bar(x + ancho/2, top_deportes_rus.values, ancho, label='Rusia (1996-2024)', color='#1f77b4', alpha=0.8)

ax.set_xlabel('Deporte', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Comparación Top 10 Deportes: URSS vs Rusia', fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(deportes_top, rotation=45, ha='right', fontsize=11)
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()


## GRÁFICO 5: Top 10 Años con Más Medallas - USA


In [ ]:
top_10_años = usa.groupby('Year').size().sort_values(ascending=False).head(10)

fig, ax = plt.subplots(figsize=(14, 7))
colores = ['orange' if año == 1904 else 'gold' if año == 1984 else 'steelblue' for año in top_10_años.index]

ax.bar(range(len(top_10_años)), top_10_años.values, color=colores, alpha=0.8, edgecolor='black', linewidth=1.5)
ax.set_xticks(range(len(top_10_años)))
ax.set_xticklabels(top_10_años.index, fontsize=11, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Top 10 Años con Más Medallas - USA', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, axis='y', alpha=0.3, linestyle='--')

for i, v in enumerate(top_10_años.values):
    ax.text(i, v + 3, str(v), ha='center', va='bottom', fontsize=11, fontweight='bold')

from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='orange', label='1904: Ventaja local'),
    Patch(facecolor='gold', label='1984: Sin URSS'),
    Patch(facecolor='steelblue', label='Otros años')
]
ax.legend(handles=legend_elements, fontsize=10, loc='upper right')
plt.tight_layout()
plt.show()


## GRÁFICO 6: Top 10 Mejores Deportes USA (Histórico)


In [ ]:
mejores_deportes_usa = usa.groupby('Sport').size().sort_values(ascending=False).head(10)

fig, ax = plt.subplots(figsize=(12, 7))
ax.barh(range(len(mejores_deportes_usa)), mejores_deportes_usa.values, color='steelblue', alpha=0.8, edgecolor='navy', linewidth=1.5)
ax.set_yticks(range(len(mejores_deportes_usa)))
ax.set_yticklabels(mejores_deportes_usa.index, fontsize=12)
ax.set_xlabel('Número de Medallas (1896-2024)', fontsize=12, fontweight='bold')
ax.set_title('Top 10 Deportes de USA (Histórico)', fontsize=15, fontweight='bold', pad=20)
ax.invert_yaxis()
ax.grid(True, axis='x', alpha=0.3, linestyle='--')

for i, v in enumerate(mejores_deportes_usa.values):
    ax.text(v + 10, i, str(v), va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()


## GRÁFICO 7: Boicots 1980 y 1984 (Detalle)


In [ ]:
años_comparacion = list(range(1968, 1992, 4))
urss_evolucion = urss[urss['Year'].isin(años_comparacion)]
medallas_urss_años = urss_evolucion.groupby('Year').size().reindex(años_comparacion, fill_value=0)
usa_periodo_urss = usa[(usa['Year'] >= 1952) & (usa['Year'] <= 1988)]
usa_evolucion = usa_periodo_urss[usa_periodo_urss['Year'].isin(años_comparacion)]
medallas_usa_años = usa_evolucion.groupby('Year').size().reindex(años_comparacion, fill_value=0)

fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(medallas_usa_años.index, medallas_usa_años.values, marker='o', linewidth=3, markersize=10, label='USA', color='#1f77b4')
ax.plot(medallas_urss_años.index, medallas_urss_años.values, marker='s', linewidth=3, markersize=10, label='URSS', color='#d62728')

ax.axvspan(1979.5, 1980.5, alpha=0.25, color='red', label='Boicot USA (Moscú 1980)')
ax.axvspan(1983.5, 1984.5, alpha=0.25, color='blue', label='Boicot URSS (LA 1984)')

ax.annotate('USA ausente\\n(0 medallas)', xy=(1980, 0), xytext=(1980, 50),
            arrowprops=dict(arrowstyle='->', color='red', lw=2), fontsize=11, ha='center', fontweight='bold', color='red')
ax.annotate('URSS ausente\\n(0 medallas)', xy=(1984, 0), xytext=(1984, 50),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2), fontsize=11, ha='center', fontweight='bold', color='blue')

ax.set_xlabel('Año Olímpico', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Guerra Fría Olímpica: Impacto de los Boicots (1968-1992)', fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(años_comparacion)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()


## GRÁFICO 8: Comparación URSS vs USA (1952-1988)


In [ ]:
años_periodo = sorted(urss['Year'].unique())
medallas_urss_periodo = urss.groupby('Year').size()
medallas_usa_periodo = usa[(usa['Year'] >= 1952) & (usa['Year'] <= 1988)].groupby('Year').size().reindex(años_periodo, fill_value=0)

fig, ax = plt.subplots(figsize=(14, 7))
x = np.arange(len(años_periodo))
ancho = 0.35

ax.bar(x - ancho/2, medallas_urss_periodo.values, ancho, label='URSS', color='#d62728', alpha=0.8)
ax.bar(x + ancho/2, medallas_usa_periodo.values, ancho, label='USA', color='#1f77b4', alpha=0.8)

ax.set_xlabel('Año Olímpico', fontsize=12, fontweight='bold')
ax.set_ylabel('Número de Medallas', fontsize=12, fontweight='bold')
ax.set_title('Rivalidad Olímpica: URSS vs USA (1952-1988)', fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(años_periodo, rotation=45, ha='right')
ax.legend(fontsize=12, loc='upper left')
ax.grid(True, axis='y', alpha=0.3, linestyle='--')

if 1980 in años_periodo:
    idx_1980 = list(años_periodo).index(1980)
    ax.annotate('Boicot USA', xy=(idx_1980, medallas_urss_periodo.iloc[idx_1980]), 
                xytext=(idx_1980, medallas_urss_periodo.iloc[idx_1980] + 30),
                arrowprops=dict(arrowstyle='->', color='red', lw=2), fontsize=10, ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f'Total URSS: {medallas_urss_periodo.sum():,} | Total USA: {medallas_usa_periodo.sum():,}')
